The BWT
-------

Follow along at doi:10.1093/bioinformatics/btp324, _Fast and accurate short read alignment with Burrows–Wheeler transform_ by Heng Li and Richard Durbin.

But note that a couple of their definitions seem to be incorrect. Adjustments will be noted.

For an alphabet of symbols:

$\Sigma = [ A, C, G, T ] $

$\alpha = $ one symbol of alphabet $\Sigma$

${X}$ = source string $\alpha_{0}\alpha_{1}\ldots\alpha_{n-1}$

$\$ =$ end of string

${n}$ = number of symbols in ${X}$

${X}[i] = \alpha_i$

${i} = 0,1,\ldots,{n-1}$

${X}[i,j] = \alpha_i\ldots\alpha_j$ (substring)

${X_i} = X[i,n-1]$ (suffix)

$S(i) = i$ th lexicographically smallest suffix (aka index into X where suffix starts)

$B$ is the BWT string: list of symbols that precede the first symbol in the sorted suffix list

>$B[i] = \$$ when $S(i) = 0$

>$B[i] = X[S(i) - 1]$

$W =$ a potential substring of $X$

Bounds:

>$\underline{R}(W) = min\{k:W $ is the prefix of $X_{S(k)}\}$

>$\overline{R}(W) = max\{k:W $ is the prefix of $X_{S(k)}\}$

For empty string $W = \$$:

>$\underline{R}(W) = 0$

>$\overline{R}(W) = n - 1$

(Note that Li and Durbin define $\underline{R}(W) = 1$ for empty string W to eliminate the need to define $O(\alpha, -1)$, but this leads to off-by-one errors later.)

Set of positions of all occurrences of $W$ in $X$:

>$\{S(k):\underline{R}(W) <= k <= \overline{R}(W)\}$

Is W a substring of X?

> If $\underline{R}(W) > \overline{R}(W)$ then $W$ is not a substring of $X$.

> If $\underline{R}(W) = \overline{R}(W)$ then $W$ matches exactly one BWT entry.

> If $\underline{R}(W) < \overline{R}(W)$ then $W$ matches all BWT entries between (inclusive).

$SA$ interval $= [ \underline{R}(W), \overline{R}(W) ]$

Backward search in $O(|W|)$ time:

>$C(\alpha) =$ # of symbols in $X[0,n-1)$ (exclusive!) that are lexicographically smaller than $\alpha$

>$O(\alpha,i) =$ # of occurrences of $\alpha$ in $B[0,i]$ (inclusive!)

By Spiral's definition:

>$O(\alpha, -1) = 0$

If $W$ is a substring of $X$:

>$\underline{R}(\alpha{W}) = C(\alpha) + O(\alpha,\underline{R}(W)-1) + 1$

>$\overline{R}(\alpha{W}) = C(\alpha) + O(\alpha, \overline{R}(W))$


In [93]:
# For string X
X = "ATTGCTAC$"

# Calculate all suffixes
suffixes = sorted([X[i:] for i in range(len(X))])

print "# suffix"
for i, suffix in enumerate(suffixes):
    print "{i} {suffix}".format(i=i, suffix=suffix)

# suffix
0 $
1 AC$
2 ATTGCTAC$
3 C$
4 CTAC$
5 GCTAC$
6 TAC$
7 TGCTAC$
8 TTGCTAC$


In [94]:
# Calculate S
S = []
for suffix in suffixes:
    S.append(X.find(suffix))
    
print S

[8, 6, 0, 7, 4, 3, 5, 2, 1]


In [95]:
# C(a) =  # of symbols in X[0,n−1) that are lexicographically smaller than a.

# Precalculate the C(a) table. This lets us look up C(a) without knowing B.
Ca = {}

# all unique symbols in X except for $
symbols = ''.join(sorted(list(set(X)))[1:])

for symbol in symbols:
    print symbol + ': ' + str([x for x in X[:-1] if x < symbol])
    Ca[symbol] = len([x for x in X[:-1] if x < symbol])

print '\n', Ca

A: []
C: ['A', 'A']
G: ['A', 'C', 'A', 'C']
T: ['A', 'G', 'C', 'A', 'C']

{'A': 0, 'C': 2, 'T': 5, 'G': 4}


In [96]:
# B: X[S(i)-1]
def B(i):
    return X[S[i]-1]

# n == |X| == |B| == |S|
n = len(X)

# String representation of B
B_str = ''.join([B(i) for i in range(n)])

print B_str
print n

CT$AGTCTA
9


In [97]:
# O(a,i): number of occurrences of a in B up to index i (inclusive)
def O(a, i):
    if i < 0:
        return 0 # O(a, -1) == 0
    count = 0
    for base in B_str[:i+1]:
        if base == a:
            count += 1
    return count

# r underbar: first suffix that matches W (silly linear search)
def r(w):
    if not w:
        return 0 # r('') == 0
    for i, suffix in enumerate(suffixes):
        if w == suffix[:len(w)]:
            return i
    return n

# R overbar: last suffix that matches W (silly linear search)
def R(w):
    if not w:
        return n - 1 # R('') = n - 1
    for i, suffix in enumerate(suffixes[::-1]):
        if w == suffix[:len(w)]:
            return n - i - 1
    return 1
    
# SA value: compute [i,j] for W
def SA(w):
    return [r(w), R(w)]

In [98]:
# Let's find SA values for some substrings
print "# suffix"
for i, suffix in enumerate(suffixes):
    print "{i} {suffix}".format(i=i, suffix=suffix)

print "\nB = " + B_str + "\n"

for symbol in symbols:
    print symbol + ':', SA(symbol)

queries = [
    'GCT', # i == j, exactly one match
    'GC', # i == j, exactly one match
    'GA',  # i > j, not in X
    'T',   # i < j, more than one match
    '',    # empty string, full range
]

for q in queries:
    print "SA('" + q + "') = " + str(SA(q))

# suffix
0 $
1 AC$
2 ATTGCTAC$
3 C$
4 CTAC$
5 GCTAC$
6 TAC$
7 TGCTAC$
8 TTGCTAC$

B = CT$AGTCTA

A: [1, 2]
C: [3, 4]
G: [5, 5]
T: [6, 8]
SA('GCT') = [5, 5]
SA('GC') = [5, 5]
SA('GA') = [9, 1]
SA('T') = [6, 8]
SA('') = [0, 8]


In [99]:
# Calculate bitcounts, saving start entry of each base
from copy import deepcopy

bitcounts = [{'A':0, 'C':0, 'G':0, 'T':0}]

for i, f in enumerate(S):
    prev = deepcopy(bitcounts[i])
    this = {}
    
    for base in "ACGT":
        this[base] = prev[base]
        
    if(f):
        base = X[f - 1]
        this[base] = this[base] + 1
        
    bitcounts.append(this)

# Drop the placeholder bitcounts[0] row
bitcounts = bitcounts[1:]

print "Bitcounts:\n"
for i, b in enumerate(bitcounts):
    print "{i} {b}".format(i=i, b=b)


Bitcounts:

0 {'A': 0, 'C': 1, 'T': 0, 'G': 0}
1 {'A': 0, 'C': 1, 'T': 1, 'G': 0}
2 {'A': 0, 'C': 1, 'T': 1, 'G': 0}
3 {'A': 1, 'C': 1, 'T': 1, 'G': 0}
4 {'A': 1, 'C': 1, 'T': 1, 'G': 1}
5 {'A': 1, 'C': 1, 'T': 2, 'G': 1}
6 {'A': 1, 'C': 2, 'T': 2, 'G': 1}
7 {'A': 1, 'C': 2, 'T': 3, 'G': 1}
8 {'A': 2, 'C': 2, 'T': 3, 'G': 1}


A little bit of bit math
------------------------

The whole point of this exercise is to quickly find the position(s) in $X$ where $W$ is an exact match (if any). One more simplification lets us calculate $O(\alpha, i)$ with bitcount lookups:

>$\underline{R}(\alpha{W}) = C(\alpha) + O(\alpha,\underline{R}(W)-1) + 1$

>$O(\alpha,\underline{R}(W)-1) = \underline{R}(\alpha{W}) - C(\alpha) - 1$

If $\underline{R}(\alpha{W})$ and $C(\alpha)$ are known, then $\underline{R}(W)-1$ becomes a lookup in the bitcount table.


In [100]:
# fast_O(a,i): lookup r(W) in the bitcounts table
def fast_O(a, i):
    if i < 0:
        return 0
    return bitcounts[i][a]

# The two methods are equivalent
for a in symbols:
    for i in range(n):
        if O(a, i) != fast_O(a, i):
            raise RuntimeError("O({0},{1}) {2} != {3}".format(a, i, O(a, i), fast_O(a, i)))

print "O(a,i) == fast_O(a,i)"

O(a,i) == fast_O(a,i)


In [101]:
# r underbar: lower limit of substring W in BWT
# by Spiral's definition, r('$') == 0
r_cache = {'': 0}
def fast_r(w):
    # Precache all substrings. We're gonna need them.
    for aW in [w[i:] for i in range(len(w))][::-1]:
        if(not aW in r_cache):
            a = aW[0]
            W = aW[1:]
            r_cache[aW] = Ca[a] + fast_O(a, fast_r(W) - 1) + 1
    return r_cache[w]

# R overbar: upper limit of substring W in BWT
# by definition, $('$') == n - 1
R_cache = {'': n - 1}
def fast_R(w):
    for aW in [w[i:] for i in range(len(w))][::-1]:
        if(not aW in R_cache):
            a = aW[0]
            W = aW[1:]
            R_cache[aW] = Ca[a] + fast_O(a, fast_R(W))
    return R_cache[w]
    
# SA value: compute [i,j] for W
def fast_SA(w):
    return [fast_r(w), fast_R(w)]

In [102]:
print "# suffix"
for i, suffix in enumerate(suffixes):
    print "{i} {suffix}".format(i=i, suffix=suffix)
print

for symbol in symbols:
    print symbol + ':', SA(symbol), fast_SA(symbol)
print 

queries = [
    'GCT', # i == j, exactly one match
    'GA',  # i > j, not in X
    'T',   # i < j, more than one match
    '',    # empty string, full range
]

for q in queries:
    print "SA('" + q + "') = " + str(SA(q)) + ' ' + str(fast_SA(q))


# suffix
0 $
1 AC$
2 ATTGCTAC$
3 C$
4 CTAC$
5 GCTAC$
6 TAC$
7 TGCTAC$
8 TTGCTAC$

A: [1, 2] [1, 2]
C: [3, 4] [3, 4]
G: [5, 5] [5, 5]
T: [6, 8] [6, 8]

SA('GCT') = [5, 5] [5, 5]
SA('GA') = [9, 1] [5, 4]
SA('T') = [6, 8] [6, 8]
SA('') = [0, 8] [0, 8]


In [124]:
# Century table: getting back to X

# Keep a position entry every (mod) positions in the original sequence
mod = 3

# Also track the cumulative count of century bits (a la bitcount)
centcount = [0]

print "Century bits:\n"
print "# c o suffix"
for i, s in enumerate(suffixes):
    centbit = 0 if S[i]%mod else 1
    centcount.append(centcount[-1] + centbit)
    print "{i} {m} {o} {s}".format(i=i, m=centbit, o=centcount[i], s=s)

century = []

for i, f in enumerate(S):
    if not S[i]%mod:
        century.append(f)

print "\nCentury table:\n"
print "# pos"
for i, c in enumerate(century):
    print "{i} {c}".format(i=i, c=c)

Century bits:

# c o suffix
0 0 0 $
1 1 0 AC$
2 1 1 ATTGCTAC$
3 0 2 C$
4 0 2 CTAC$
5 1 2 GCTAC$
6 0 3 TAC$
7 0 3 TGCTAC$
8 0 3 TTGCTAC$

Century table:

# pos
0 6
1 0
2 3


In [125]:
def w_to_x(W):
    (i, j) = fast_SA(W)

    reply = []
    
    for k in range(i, j + 1):
        e = k  # e is the bwt entry under examination 
        w = W  # we will be pushing bases to the front of w
        d = 0  # distance from century entry (# of pushes)

        # No need to store the full century table: if centcount goes up on the next entry,
        # then this is a century entry.
        while centcount[e + 1] - centcount[e] == 0:
            w = B_str[e] + w
            e = fast_r(w)
            d += 1
        reply.append(century[centcount[e]] + d)
        
    return sorted(reply)

def find_me(W):
    print '{}:'.format(W)
    for pos in w_to_x(W):
        if W != X[pos:pos + len(W)]:
            raise RuntimeError("X[{}] {} != {}".format(pos, X[pos:pos + len(W)], W))
        print "  X[{}] == {}".format(pos, X[pos:pos + len(W)])


In [126]:
print "X = {}\n".format(X)

print 'All symbols:'
for base in symbols:
    find_me(base)
    print
    
queries = [
    'GCT',
    'AAA',
    'TGCTAC',
    'ATT',
    'TGCTA',
    'CTA',
    'CTTAGGAGAAC',
    'AC'
]

print 'Canned lookups:'
for q in queries:
    find_me(q)
    print

X = ATTGCTAC$

All symbols:
A:
  X[0] == A
  X[6] == A

C:
  X[4] == C
  X[7] == C

G:
  X[3] == G

T:
  X[1] == T
  X[2] == T
  X[5] == T

Canned lookups:
GCT:
  X[3] == GCT

AAA:

TGCTAC:
  X[2] == TGCTAC

ATT:
  X[0] == ATT

TGCTA:
  X[2] == TGCTA

CTA:
  X[4] == CTA

CTTAGGAGAAC:

AC:
  X[6] == AC



In [127]:
def revcomp(seq):
    flip = {
        'A': 'T',
        'C': 'G',
        'G': 'C',
        'T': 'A',
        'N': 'N',
        'a': 't',
        'c': 'g',
        'g': 'c',
        't': 'a',
        'n': 'n',
        '$': ''
    }

    reply = ''
    for c in seq:
        reply += flip[c]

    # Got a $? Leave a $.
    if c == '$':
        reply = '$' + reply

    return reply[::-1]

def find_me_twice(W):
    print 'Query {}:'.format(W)
    for pos in w_to_x(W):
        if W != X[pos:pos + len(W)]:
            raise RuntimeError("X[{}] {} != {}".format(pos, X[pos:pos + len(W)], W))
        print "  X[{}] >> {}".format(pos, X[pos:pos + len(W)])

    for pos in w_to_x(revcomp(W)):
        if revcomp(W) != X[pos:pos + len(W)]:
            raise RuntimeError("X[{}] {} != {}".format(pos, revcomp(W), X[pos:pos + len(W)]))

        rpos = n - 1 - pos - len(W) # Position in reverse complement(X)
        print "  R[{}] == X[{}] << {}".format(rpos, pos, revcomp(X)[rpos:rpos + len(W)])
            
print X
print revcomp(X)
print 
find_me_twice('T')

ATTGCTAC$
GTAGCAAT$

Query T:
  X[1] >> T
  X[2] >> T
  X[5] >> T
  R[7] == X[0] << T
  R[1] == X[6] << T


In [128]:
from random import randrange, choice
runs = 10

print '{} random substring lookups:'.format(runs)
for i in range(runs):
    r = randrange(0, n - 2)
    cache = []
    q = 'x'
    while q not in cache:
        q = X[r:randrange(r + 1, n - 1)]
        cache.append(q)
    find_me_twice(q)
    
print '\n{} random 2-mer lookups:'.format(runs)
for i in range(runs):
    cache = []
    q = 'x'
    while q not in cache:
        q = choice(symbols) + choice(symbols)
        cache.append(q)
    find_me_twice(q)

print '\n{} random 3-mer lookups:'.format(runs)
for i in range(runs):
    cache = []
    q = 'x'
    while q not in cache:
        q = choice(symbols) + choice(symbols) + choice(symbols)
        cache.append(q)
    find_me_twice(q)

10 random substring lookups:
Query TTGCTA:
  X[1] >> TTGCTA
Query A:
  X[0] >> A
  X[6] >> A
  R[6] == X[1] << A
  R[5] == X[2] << A
  R[2] == X[5] << A
Query ATTG:
  X[0] >> ATTG
Query T:
  X[1] >> T
  X[2] >> T
  X[5] >> T
  R[7] == X[0] << T
  R[1] == X[6] << T
Query T:
  X[1] >> T
  X[2] >> T
  X[5] >> T
  R[7] == X[0] << T
  R[1] == X[6] << T
Query A:
  X[0] >> A
  X[6] >> A
  R[6] == X[1] << A
  R[5] == X[2] << A
  R[2] == X[5] << A
Query T:
  X[1] >> T
  X[2] >> T
  X[5] >> T
  R[7] == X[0] << T
  R[1] == X[6] << T
Query A:
  X[0] >> A
  X[6] >> A
  R[6] == X[1] << A
  R[5] == X[2] << A
  R[2] == X[5] << A
Query T:
  X[1] >> T
  X[2] >> T
  X[5] >> T
  R[7] == X[0] << T
  R[1] == X[6] << T
Query A:
  X[0] >> A
  X[6] >> A
  R[6] == X[1] << A
  R[5] == X[2] << A
  R[2] == X[5] << A

10 random 2-mer lookups:
Query AT:
  X[0] >> AT
  R[6] == X[0] << AT
Query AG:
  R[2] == X[4] << AG
Query AT:
  X[0] >> AT
  R[6] == X[0] << AT
Query TC:
Query GA:
Query AT:
  X[0] >> AT
  R[6] == X[0